In [6]:
# !wget https://huggingface.co/datasets/mesolitica/chatgpt-malaysian-qa-choice/resolve/main/qa-ms-wikipedia.jsonl

In [14]:
import json
import random
import re

def clean(string):
    string = re.sub('[^A-Za-z ]+', ' ', string.lower())
    string = re.sub(r'[ ]+', ' ', string).strip()
    return string

def overlap(string1, string2):
    l = set([w for w in clean(string1).split() if len(w) > 2])
    r = set([w for w in clean(string2).split() if len(w) > 2])
    return len(l & r) / len(l)

In [8]:
data = []
with open('qa-ms-wikipedia.jsonl') as fopen:
    for l in fopen:
        data.append(json.loads(l))

In [9]:
len(data)

8217

In [10]:
data[0]

{'paragraph': 'pertanian yang utama adalah jagung, kapas, tembakau, gandum, kopi dan tebu. Zimbabwe adalah kerajaan berpusat dan terbahagi kepada lapan wilayah dan dua bandar raya dengan status wilayah untuk tujuan pentadbiran. Setiap wilayah pula mempunyai ibu negerinya yang tersendiri di mana perniagaan rasmi dijalankan. Nama-nama sebahagian besar wilayah dihasilkan daripada perpecahan Mashonaland dan Matabeleland semasa penjajahan: Mashonaland adalah wilayah yang pertama sekali diduduki Syarikat Afrika Selatan British Pioneer Column dan Matabeleland pula ialah wilayah yang ditakluki semasa Perang Matabele Pertama. Ini sejajar dengan wilayah pra-kolonial orang Shona dan orang Matabele, walaupun terdapat etnik minoriti yang signifikan di kebanyakan wilayah. Setiap provinsi diketuai oleh seorang Gabenor Wilayah, yang dilantik oleh Presiden. Pemerintah wilayah dikendalikan oleh Pentadbir Provinsi, yang dilantik oleh Suruhanjaya Perkhidmatan Awam. Fungsi pemerintahan lain di tingkat prov

In [11]:
indices = {i for i in range(len(data))}

In [15]:
from tqdm import tqdm

pairs = []
for i in tqdm(range(len(data))):
    query = data[i]['paragraph']
    
    for s in data[i]['qa']['qa']:
        pos = [query]
        sampled = random.sample(list(indices - {i}), 100)
        neg = []
        for k in sampled:
            try:
                overlapped = overlap(query, data[k]['paragraph'])
                if overlapped < 0.3:
                    neg.append(data[k]['paragraph'])
                if len(neg) >= 5:
                    break
            except:
                pass
            
        pairs.append({
            'query': s['question'],
            'pos': pos,
            'neg': neg
        })

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8217/8217 [02:29<00:00, 55.11it/s]


In [19]:
len(pairs)

41793

In [20]:
pairs[0]

{'query': 'Apakah nama-nama sebahagian besar wilayah di Zimbabwe?',
 'pos': ['pertanian yang utama adalah jagung, kapas, tembakau, gandum, kopi dan tebu. Zimbabwe adalah kerajaan berpusat dan terbahagi kepada lapan wilayah dan dua bandar raya dengan status wilayah untuk tujuan pentadbiran. Setiap wilayah pula mempunyai ibu negerinya yang tersendiri di mana perniagaan rasmi dijalankan. Nama-nama sebahagian besar wilayah dihasilkan daripada perpecahan Mashonaland dan Matabeleland semasa penjajahan: Mashonaland adalah wilayah yang pertama sekali diduduki Syarikat Afrika Selatan British Pioneer Column dan Matabeleland pula ialah wilayah yang ditakluki semasa Perang Matabele Pertama. Ini sejajar dengan wilayah pra-kolonial orang Shona dan orang Matabele, walaupun terdapat etnik minoriti yang signifikan di kebanyakan wilayah. Setiap provinsi diketuai oleh seorang Gabenor Wilayah, yang dilantik oleh Presiden. Pemerintah wilayah dikendalikan oleh Pentadbir Provinsi, yang dilantik oleh Suruhanj

In [17]:
with open('qa-ms-wikipedia-pair.jsonl', 'w') as fopen:
    for l in pairs:
        if not len(l['pos']) or not len(l['neg']):
            continue
        fopen.write(f'{json.dumps(l)}\n')

In [18]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj='qa-ms-wikipedia-pair.jsonl',
    path_in_repo='qa-ms-wikipedia-pair.jsonl',
    repo_id='mesolitica/instructions-pair-mining',
    repo_type='dataset',
)

qa-ms-wikipedia-pair.jsonl:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/instructions-pair-mining/commit/5c6c23993280207d08cafa5038d65519aafb091f', commit_message='Upload qa-ms-wikipedia-pair.jsonl with huggingface_hub', commit_description='', oid='5c6c23993280207d08cafa5038d65519aafb091f', pr_url=None, pr_revision=None, pr_num=None)